In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<img src="./images/nvmath_head_panel@0.5x.png" alt="nvmath-python" />

# Getting Started with nvmath-python: FFT Callbacks

## Exercise: Applying Sepia and Gaussian blur filters to an image

In this exercise you will perform the following steps:
1. Load the color image using `PIL`.
2. Create and compile the *Sepia* filter for the R2C FFT prolog 
3. Create and compile the *Gaussian Blur* filter for the C2R FFT prolog.
4. Perform forward FFT with the compiled Sepia filter prolog.
5. Perform backward C2R FFT with the compiled Gaussian Blur filter prolog.
6. Display the processed image using `matplotlib`

The following NumPy implementation is a reference code for you to follow while implementing nvmath-python variant with forward FFT prolog and epilog:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# 1. Load the color image using PIL
# Load the color image and convert it to NumPyarray normalized to [0, 1]
asset_path = "./images/"
original_image = np.array(Image.open(asset_path + "dog.jpg")) / 255.0


# Display the original image
plt.figure(figsize=(10, 5))

plt.imshow(original_image)
plt.title("Original Image")
plt.axis("off")

plt.tight_layout()
plt.show()

original_image.shape

Note the shape of the array: spatial data goes in dimensions 0 and 1, followed by color channels data.

The following code is a reference implementation of the Sepia filter, which is a color matrix transformation applied to original pixels capped by 1.0.

In [ ]:
SEPIA_FILTER_MATRIX = np.array([
    [0.393, 0.769, 0.189],
    [0.349, 0.686, 0.168],
    [0.272, 0.534, 0.131]
]).T


def sepia_filter(image):
    """
    Apply sepia filter to the image.
    Image shape: (height, width, 3)
    Returns: (height, width, 3)
    """
    # Apply filter: (height*width, 3) @ (3, 3) = (height*width, 3)
    return np.minimum(1.0, image @ SEPIA_FILTER_MATRIX)


sepia_image = sepia_filter(original_image)

# Display the sepia image
plt.figure(figsize=(10, 5))

plt.imshow(sepia_image)
plt.title("Sepia Image")
plt.axis("off")

plt.tight_layout()
plt.show()




Finally we apply Gaussian Blur filter implemented through R2C and C2R FFTs:

In [ ]:
sigma = 20.0
fy = np.fft.fftfreq(sepia_image.shape[0])[:, None]  # column vector
fx = np.fft.rfftfreq(sepia_image.shape[1])[None, :]  # row vector for R2C (only positive frequencies)
h = np.exp(-2.0 * np.pi * np.pi * sigma * sigma * (fx * fx + fy * fy)).astype(np.complex64)
print(h.shape)


def gaussian_filter_numpy(image):
    # Apply FFT on spatial dimensions (axes 0 and 1), leaving color channel intact
    image_fft = np.fft.rfft2(image, axes=(0, 1))  # Real to complex FFT
    blurred = np.fft.irfft2(image_fft * h[..., None], axes=(0, 1))  # Complex to real FFT
    return blurred

blurred_image = gaussian_filter_numpy(sepia_image)

# Display the blurred image
plt.figure(figsize=(10, 5))

plt.imshow(blurred_image)
plt.title("Sepia & Blurred Image")
plt.axis("off")

plt.tight_layout()
plt.show()


Note the shape of `h`: for R2C FFT we neeed only positive frequences, hence the size in `fx` dimension is `image.shape[1] // 2 + 1`.

Now porting the NumPy implementation to GPU with CuPy

In [ ]:
import nvmath # Workaround for CuPy: CTK shared objects preload from wheels
import cupy as cp

image_gpu = cp.asarray(original_image, dtype=cp.float32)
SEPIA_FILTER_MATRIX_GPU = cp.asarray(SEPIA_FILTER_MATRIX).astype(cp.float32)


def sepia_filter(image):
    return cp.minimum(1.0, image @ SEPIA_FILTER_MATRIX_GPU)


sepia_image = sepia_filter(image_gpu)

# Display the sepia image
plt.figure(figsize=(10, 5))

plt.imshow(sepia_image.get())
plt.title("Sepia Image")
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
sigma = 20.0
fy = cp.fft.fftfreq(sepia_image.shape[0])[:, None].astype(cp.float32)  # column vector
fx = cp.fft.rfftfreq(sepia_image.shape[1])[None, :].astype(cp.float32)  # row vector for R2C (only positive frequencies)
h = cp.exp(-2.0 * np.pi * np.pi * sigma * sigma * (fx * fx + fy * fy)).astype(cp.complex64)


def gaussian_filter_cupy(image):
    # Apply FFT on spatial dimensions (axes 0 and 1), leaving color channel intact
    image_fft = cp.fft.rfft2(image, axes=(0, 1))  # Real to complex FFT
    blurred = cp.fft.irfft2(image_fft * h[..., None], axes=(0, 1))  # Complex to real FFT
    return blurred

blurred_image = gaussian_filter_cupy(sepia_image)

# Display the blurred image
plt.figure(figsize=(10, 5))

plt.imshow(blurred_image.get())
plt.title("Sepia & Blurred Image")
plt.axis("off")

plt.tight_layout()
plt.show()


Finally, we get to implement the above code using nvmath-python with compiled prolog and epilog functions

In [ ]:
# Dimensions over which FFT is performed must be contiguous in memory
image_gpu = cp.asarray(original_image, dtype=cp.float32).transpose(2, 0, 1)
wh = image_gpu.shape[1] * image_gpu.shape[2]
h_size = h.size

def sepia_prolog_impl(data_in, offset, user_info, unused):
    channel_idx = offset % 3
    pixel_idx = offset - channel_idx
    r = data_in[pixel_idx]
    g = data_in[pixel_idx + 1]
    b = data_in[pixel_idx + 2]
    if channel_idx == 0:
        new_r = r * 0.393 + g * 0.769 + b * 0.189
        return min(1.0, new_r)
    elif channel_idx == 1:
        new_g = r * 0.349 + g * 0.686 + b * 0.168
        return min(1.0, new_g)
    else:
        new_b = r * 0.272 + g * 0.534 + b * 0.131
        return min(1.0, new_b)


def blur_prolog_impl(data_in, offset, filter_data, unused):
    # Offset is a flattened index of the three (R, G, B) FFTs. Each FFT has size h.size 
    return data_in[offset] * filter_data[offset % h_size] / wh  # Normalize by the image area

sepia_prolog = nvmath.fft.compile_prolog(sepia_prolog_impl, "float32", "float32")
blur_prolog = nvmath.fft.compile_prolog(blur_prolog_impl, "complex64", "complex64")

fft_image = nvmath.fft.rfft(image_gpu, axes=(1, 2), prolog={"ltoir": sepia_prolog, "data": None})
filtered_image = nvmath.fft.irfft(fft_image, axes=(1, 2), prolog={"ltoir": blur_prolog, "data": h.data.ptr})

plt.figure(figsize=(10, 5))

plt.imshow(filtered_image.transpose(1, 2, 0).get())
plt.title("Sepia & Blurred Image")
plt.axis("off")

plt.tight_layout()
plt.show()